In [1]:
import pandas as pd 
import numpy as np 
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [94]:
data = pd.read_csv("./Data_Clean.csv", index_col=0)

In [95]:
data = data.dropna()

In [96]:
data = data[data['vote_count'] > 300] 

In [98]:
data = data.reset_index(drop=True)

In [100]:
for idx in data.index:
    data.at[idx, 'director'] = ast.literal_eval(data.at[idx, 'director'])
    data.at[idx, 'screenplay'] = ast.literal_eval(data.at[idx, 'screenplay'])
    data.at[idx, 'genres'] = ast.literal_eval(data.at[idx, 'genres'])
    data.at[idx, 'producer'] = ast.literal_eval(data.at[idx, 'producer'])

In [101]:
def replace_spaces(idx, category):
    names = data.at[idx, category]
    for idx in range(len(names)):
        new = names[idx].replace(' ', '_')
        names[idx] = new
    return names

In [102]:
for idx in data.index:
    data.at[idx, 'director'] = replace_spaces(idx, 'director')
    data.at[idx, 'screenplay'] = replace_spaces(idx, 'screenplay')
    data.at[idx, 'producer'] = replace_spaces(idx, 'producer')

In [104]:
for idx in data.index:
    data.at[idx, 'info'] = ' '.join(data.at[idx, 'director']) + ' ' + ' '.join(data.at[idx, 'producer']) + ' ' + ' '.join(data.at[idx, 'screenplay']) + ' ' + ' '.join(data.at[idx, 'genres'])

In [105]:
cv = CountVectorizer(stop_words='english')
matrix = cv.fit_transform(data['info'])

In [106]:
cos_sim = cosine_similarity(matrix, matrix)

In [107]:
idxs = pd.Series(data.index, index=data['title'])

In [108]:
def recommendations(title, cos_sim=cos_sim):
    idx = idxs[title]
    scores = list(enumerate(cos_sim[idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    scores = scores[1:11]
    movie_idxs = [i[0] for i in scores]
    return data['title'].iloc[movie_idxs]

In [111]:
recommendations('Skyfall')

364         Tomorrow Never Dies
623     The World Is Not Enough
944             Die Another Day
3                     GoldenEye
1403              Casino Royale
1630          Quantum of Solace
2730                    Spectre
491            A View to a Kill
607             Licence to Kill
765        The Living Daylights
Name: title, dtype: object